In [ ]:
%pip install torch numpy transformers datasets huggingface

### Importing the Required libraries

Hugginface key is required 

[Huggingface](https://huggingface.co/)

use the link to signup and create a api key free of cost


In [ ]:
from huggingface_hub import login
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset
huggingfacekey = "hugginface_key"
login(token=huggingfacekey)

c:\Users\ajukh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\ajukh\.cache\huggingface\token
Login successful


## Random Index Generation and Dataset Loading Example

### Random Index Generation:
    - Uses np.random.randint() to generate random integers.
    - Generates n = 10 random integers between 0 and 30000.
### CNN/DailyMail Dataset:
    - Loaded using load_dataset("cnn_dailymail", "3.0.0").
    - Contains articles and summaries used for NLP tasks such as summarization.


In [ ]:
import numpy as np
n = 10
randindex = np.random.randint(30000,size=10)
print(randindex)
dataset = load_dataset("cnn_dailymail", "3.0.0")
print(dataset)

##### Selecting Random Articles , BART Model

Random indices are generated, and the corresponding articles from the dataset are selected.


The BART (Bidirectional and Auto-Regressive Transformer) model is loaded using Hugging Face.
This model is used for text summarization.

##### Hugging Face Pretrained Models:

The "facebook/bart-large-cnn" model is used for summarizing news articles.

In [ ]:
# Extracting a subset of articles from the 'train' dataset using random indices stored in 'randindex'
articles = dataset['train'][randindex]['article']

# Load the pre-trained BART model and tokenizer
# The 'facebook/bart-large-cnn' is a model specifically fine-tuned for text summarization tasks
model_name = "facebook/bart-large-cnn"

# Loading the BART tokenizer, which is used to encode and decode text for the model
tokenizer = BartTokenizer.from_pretrained(model_name)

# Loading the pre-trained BART model, which is used for generating summaries
model = BartForConditionalGeneration.from_pretrained(model_name)


c:\Users\ajukh\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ajukh\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\ajukh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\

**Purpose:** This code defines a function summarize_text() that takes a text input and generates a summary using a pre-trained transformer model like BART.

**Inputs:** The function accepts a text string and processes it with the tokenizer to convert it into a format suitable for the model. The input text is truncated to a maximum length of 1024 tokens to fit within the model's constraints.

**Model Summarization:** It uses the model’s generate() method to create a summary with specific parameters like max_length, min_length, and beam search (num_beams=4) for optimal summary quality. Early stopping is applied to halt the generation process once a suitable summary is reached.

**Output:** The function decodes the generated token IDs into a readable text summary and returns it, excluding any special tokens.

In [ ]:
# Function to summarize text using a pre-trained model
def summarize_text(text):
    # Tokenizing the input text, truncating to a max length of 1024 tokens, and padding if necessary
    # The 'return_tensors' argument ensures the output is in a format compatible with PyTorch
    inputs = tokenizer(text, max_length=1024, return_tensors="pt", truncation=True, padding=True)

    # Generating the summary using the model
    # max_length=150 ensures the summary is not longer than 150 tokens
    # min_length=40 ensures the summary is at least 40 tokens long
    # length_penalty=2.0 penalizes longer summaries, encouraging conciseness
    # num_beams=4 specifies beam search with 4 beams for better exploration of possible summary outputs
    # early_stopping=True stops the beam search once the best summary is found
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decoding the generated summary (converting token IDs back to text)
    # skip_special_tokens=True ensures special tokens like padding and end-of-sequence tokens are excluded
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Returning the summarized text
    return summary


In [21]:
import requests
from bs4 import BeautifulSoup
url = "https://en.wikipedia.org/wiki/Fine-tuning_(deep_learning)"

def scraping(url):
    # Define browser-like headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }

    try:
        # Sending request to get the data from the webpage
        scraped_data = requests.get(url, headers=headers)
        scraped_data.raise_for_status()  # Check if request was successful (status code 200)

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(scraped_data.content, 'html.parser')

        # Find all paragraphs and extract text
        paragraphs = soup.find_all('p')
        article_context = ""

        for p in paragraphs:
            article_context += p.get_text(strip=True) + " "  # Strip whitespace and concatenate paragraphs

        return article_context.strip()  # Return the article content, ensuring no leading/trailing whitespace

    except requests.exceptions.RequestException as e:
        # Handle any exception raised by requests
        print(f"An error occurred: {e}")
        return None

In [23]:
# for i , article in enumerate(articles):
#     summary = summarize_text(article)
#     print(f"Original Article {i+1}:\n{article}\n")
#     print(f"Summary {i+1}:\n{summary}\n")
#     print("-" * 80)
#     break
article = scraping(url)
summary = summarize_text(article)
def WriteFile(final_summary):
    with open('./summary.txt',mode='a', encoding='utf-8') as file:
        file.write(f"\n \n \n The LLM (BART) summarized text  is as follows : \n {final_summary}")
WriteFile(summary)